# Cross-Batch Memory for Embedding Learning


* URL: https://arxiv.org/abs/1912.06798
* authors: Xun Wang, Haozhi Zhang, Weilin Huang, Matthew R. Scott
* Submitted on 14 Dec 2019 (v1)

## 1. どんなもの？
XBM mechanism, where stands for Cross-Batch Memoryの提案。 "Slow drift phenomena"を利用することで過去のイテレーションのembeddingを記憶しハードネガティブなペアを見つけることができる。

## 2. 先行研究と比べるとどこが凄い？  
それぞれのmini-batch内でネガティブなものを選んでいた先行研究と比べて、複数のmini-batchから選べるようにすることで精度を上昇させることができた。Memory消費もほぼ0に近い。

## 3. 技術や手法のキモはどこ？
### Slow drift Phenomena
過去のmini-batchのembeddingを利用するために、異なるイテレーションで訓練されたfeatureがどれだけ変わるかを以下の式で表した。t-th iteration with $\Delta t$のときのxのthe feature driftは

$$
D(x, t;\Delta t) = ||f(\mathbf{x};\mathbf{\theta}^{t}-f(\mathbf{x};\mathbf{\theta}^{t-\Delta t}))||^2_2 \\
\text{ , where } f(\cdot;\theta)\text{ projects a data point }x_{i}\text{into a }D\text{-dimentional unit hyper-sphere}
$$

GoogleNetをContrastive lossで0から訓練し異なるiterationのときのthe feature driftの平均を計算する。iterationが小さいとき(e.g.10)には小さいが、大きいときには(e.g. 100 and 1000)早い段階で劇的に変化する。しかし3K iterationほどで安定する。さらに学習率を下げると変化の度合いも小さくなる。このような減少をslow driftとした。

### XBM
the feature driftはepochが早い段階では大きいのでwarmupとして1k iteration訓練する。その後にmini-batchのembeddingとlabelをqueueに追加し、dequeueする。??mini-batchのそれぞれのインスタンスがメモリーの中の全てのインスタンスと比較されてもっとも情報量の多いペアを見つけることができる。??
以下の実験でqueueのmemory ratio(メモリーサイズ/全体の訓練データのサイズ)は1％程度で飽和することが分かった。

## 4. どうやって有効だと検証した？
Stanford Online Products(SOP), In-shop Clothes Retrieval(In-shop), PKU VehicleID(VehicleID)の3つのdatasetのrecall@kでSOTA。

## 5. 議論はある？
mini-batchのそれぞれのインスタンスがメモリーの中の全てのインスタンスと比較されてもっとも情報量の多いペアを見つけることができる。のプロセスがいまいち分からない。  
->普通に全部とコサイン類似度を比較しているのかな。
以下公式実装より

```python
# trainer.py
images = images.to(device)
targets = targets.to(device)
feats = model(images)

if cfg.XBM.ENABLE and iteration > cfg.XBM.START_ITERATION:
    xbm.enqueue_dequeue(feats.detach(), targets.detach())

loss = criterion(feats, targets, feats, targets)
log_info["batch_loss"] = loss.item()

if cfg.XBM.ENABLE and iteration > cfg.XBM.START_ITERATION:
    xbm_feats, xbm_targets = xbm.get()
    xbm_loss = criterion(feats, targets, xbm_feats, xbm_targets)
    log_info["xbm_loss"] = xbm_loss.item()
    loss = loss + cfg.XBM.WEIGHT * xbm_loss
```

```python
# Contrastive_loss.py
@LOSS.register("contrastive_loss")
class ContrastiveLoss(nn.Module):
    def __init__(self, cfg):
        super(ContrastiveLoss, self).__init__()
        self.margin = 0.5

    def forward(self, inputs_col, targets_col, inputs_row, target_row):

        n = inputs_col.size(0)
        # Compute similarity matrix
        sim_mat = torch.matmul(inputs_col, inputs_row.t())
        epsilon = 1e-5
        loss = list()

        neg_count = list()
        for i in range(n):
            pos_pair_ = torch.masked_select(sim_mat[i], targets_col[i] == target_row)
            pos_pair_ = torch.masked_select(pos_pair_, pos_pair_ < 1 - epsilon)
            neg_pair_ = torch.masked_select(sim_mat[i], targets_col[i] != target_row)

            neg_pair = torch.masked_select(neg_pair_, neg_pair_ > self.margin)

            pos_loss = torch.sum(-pos_pair_ + 1)
            if len(neg_pair) > 0:
                neg_loss = torch.sum(neg_pair)
                neg_count.append(len(neg_pair))
            else:
                neg_loss = 0

            loss.append(pos_loss + neg_loss)
        if inputs_col.shape[0] == inputs_row.shape[0]:
            prefix = "batch_"
        else:
            prefix = "memory_"
        if len(neg_count) != 0:
            log_info[f"{prefix}average_neg"] = sum(neg_count) / len(neg_count)
        else:
            log_info[f"{prefix}average_neg"] = 0
        log_info[f"{prefix}non_zero"] = len(neg_count)
        loss = sum(loss) / n  # / all_targets.shape[1]
        return loss


# xbm.py
class XBM:
    def __init__(self, cfg):
        self.K = cfg.XBM.SIZE
        self.feats = torch.zeros(self.K, 128).cuda()
        self.targets = torch.zeros(self.K, dtype=torch.long).cuda()
        self.ptr = 0

    @property
    def is_full(self):
        return self.targets[-1].item() != 0

    def get(self):
        if self.is_full:
            return self.feats, self.targets
        else:
            return self.feats[:self.ptr], self.targets[:self.ptr]

    def enqueue_dequeue(self, feats, targets):
        q_size = len(targets)
        if self.ptr + q_size > self.K:
            self.feats[-q_size:] = feats
            self.targets[-q_size:] = targets
            self.ptr = 0
        else:
            self.feats[self.ptr: self.ptr + q_size] = feats
            self.targets[self.ptr: self.ptr + q_size] = targets
            self.ptr += q_size

## 6. 次に読むべき論文は？
* Multi-Similarity Loss with General Pair Weighting for Deep Metric Learning

## 7. 実装
https://github.com/MalongTech/research-xbm